<a href="https://colab.research.google.com/github/Janani-Withana/CTSE_Chatbot/blob/main/CTSE_Chatbot_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install transformers langchain langchain-community faiss-cpu sentence-transformers pypdf PyPDF2 langchain-huggingface

In [ ]:
# Import modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import os

In [ ]:
# Load the CTSE lecture notes PDF
pdf_path = "CTSE_Lecture_Notes.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [ ]:
# Split text into chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

In [ ]:
# Create embeddings and store in FAISS
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)

In [ ]:
# Initialize GPT-2 and setup text generation pipeline
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Ensure GPT-2 uses padding token if missing
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

generation_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    device=0 if torch.cuda.is_available() else -1,
    truncation=True
)

#llm = HuggingFacePipeline(pipeline=generation_pipeline)

In [ ]:
# Generate Response
def CTSE_Chatbot(question, top_k=5, max_new_tokens=128):
    results = vectorstore.similarity_search(question, k=top_k)
    context = "\n\n".join([doc.page_content for doc in results])

    prompt = f"""
Context:
{context}

Question:
{question}

Answer:
"""

    #print("Prompt before sending to GPU:", prompt)
    response = generation_pipeline(prompt, max_new_tokens=max_new_tokens, do_sample=True, top_p=0.9, temperature=0.7)[0]['generated_text']

    # Extract only the answer part (everything after 'Answer:')
    if "Answer:" in response:
        return response.split("Answer:")[-1].strip()
    else:
        return response.strip()

In [ ]:
# Example
question = "what are the benifits of Monolithic Architecture"
response = CTSE_Chatbot(question)

print("Question:", question + "\n")
print("Answer:", response)